In [1]:
!pip install langchain langchain-community pypdf langchain_nvidia_ai_endpoints

In [2]:
import os
os.environ['NVIDIA_API_KEY'] = "enter your api key"

In [3]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

llm = ChatNVIDIA(
  model="meta/llama-3.3-70b-instruct",
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024)

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are expert in motivating someone."),
    ("human","Motivate me by {quotes_count} motivational quotes.")
])

#Simple Chain

In [ ]:
chain = prompt_template | llm | StrOutputParser()

In [ ]:
result = chain.invoke({"quotes_count":4})

In [ ]:
print(result)

I'd be delighted to motivate you with these 4 powerful quotes:

1. **"Believe you can and you're halfway there."** - Theodore Roosevelt

Remember, your mindset is the key to unlocking your potential. Believe in yourself, your abilities, and your dreams. You've got this!

2. **"It does not matter how slowly you go as long as you do not stop."** - Confucius

Don't be discouraged by setbacks or slow progress. Every step forward, no matter how small, brings you closer to your goal. Keep moving, and you'll eventually reach your destination.

3. **"Success is not final, failure is not fatal: It is the courage to continue that counts."** - Winston Churchill

Don't be afraid to take risks and face challenges. It's the courage to keep going, even in the face of adversity, that will ultimately lead you to success. Learn from your failures and use them as stepping stones to growth.

4. **"You miss 100% of the shots you don't take."** - Wayne Gretzky

Don't let fear or doubt hold you back from pur

#What inside a Chain

In [4]:
from langchain.schema.runnable import RunnableLambda,RunnableSequence,RunnableParallel, RunnableBranch

format_prompt = RunnableLambda(lambda x:prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

In [ ]:
chain = RunnableSequence(first = format_prompt,middle = [invoke_model], last = parse_output)

In [ ]:
result = chain.invoke({"quotes_count":3})
print(result)

I'd be delighted to motivate you with these 3 powerful quotes:

1. **"Believe you can and you're halfway there."** - Theodore Roosevelt

Remember, having faith in yourself and your abilities is the first step to achieving greatness. Believe in your potential, and you'll be amazed at what you can accomplish!

2. **"It does not matter how slowly you go as long as you do not stop."** - Confucius

Don't be discouraged by setbacks or slow progress. Every step forward, no matter how small, brings you closer to your goal. Keep moving, and you'll eventually reach your destination!

3. **"You miss 100% of the shots you don't take."** - Wayne Gretzky

Don't let fear or doubt hold you back from taking risks and pursuing your dreams. Remember, every opportunity you seize is a chance to learn, grow, and succeed. Take the shot, and you might just score big!

Feel motivated yet?


#Sequential Chain

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are a good story teller about different places of the world in language {language}."),
    ("human","Tell me a story about {place_name}")
])

In [ ]:
#for_translation = RunnableLambda(lambda x: {"content":x["content"]})
for_translation = RunnableLambda(lambda output: {"content": output})
prompt_template_1 = ChatPromptTemplate.from_messages([
    ("system","You are a good language converter. you can convert any language into English language."),
    ("human","Convert the text {content}")
])

In [ ]:
chain = prompt_template | llm | StrOutputParser() | for_translation | prompt_template_1 | llm | StrOutputParser()
#chain1 = prompt_template_1 | llm | StrOutputParser()

In [ ]:
result = chain.invoke({"language":"French","place_name":"Delhi"})
print(result)
#result1 = chain1.invoke(result)
#print(result1)

Here is the translation of the text from French to English:

Delhi, the city with a thousand faces! Let me take you on a journey through the lively streets and historic monuments of this fascinating city.

Once upon a time, in the heart of India, there was a city that had seen the birth and disappearance of empires, dynasties, and civilizations. Delhi, the capital of India, is a city that breathes history, culture, and diversity. From the Mughal period to the British colonial era, each era has left its mark on the city.

Imagine yourself walking through the narrow streets of old Delhi, surrounded by mosques, temples, and colorful bazaars. The smells of spices and food surround you, while the sounds of horns, vendors' cries, and prayer chants fill the air. You feel like a traveler in a tale from the Arabian Nights, with its treasure markets and hidden secrets.

We will visit the Red Fort, that magnificent Mughal palace that was the seat of imperial power for centuries. The red sandstone

#Parallel Chain

In [ ]:
# Define prompt template for movie summary
summary_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic."),
        ("human", "Provide a brief summary of the movie {movie_name}."),
    ]
)
# Define plot analysis step
def analyze_plot(plot):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
        ]
    )
    return plot_template.format_prompt(plot=plot)

# Define character analysis step
def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the characters: {characters}. What are their strengths and weaknesses?"),
        ]
    )
    return character_template.format_prompt(characters=characters)

# Combine analyses into a final verdict
def combine_verdicts(plot_analysis, character_analysis):
    return f"Plot Analysis:\n{plot_analysis}\n\nCharacter Analysis:\n{character_analysis}"

# Simplify branches with LCEL
plot_branch_chain = (
    RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser()
)

character_branch_chain = (
    RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    summary_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches={"plot": plot_branch_chain, "characters": character_branch_chain})
    | RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"], x["branches"]["characters"]))
)

# Run the chain
result = chain.invoke({"movie_name": "Inception"})

print(result)

Plot Analysis:
**Strengths:**

1. **Original and Complex Plot**: The movie's narrative is a masterclass in storytelling, with a unique concept that explores the idea of shared dreaming and the blurring of reality and fantasy. The multiple levels of dreams within dreams create a sense of depth and complexity that keeps the audience engaged.
2. **Well-Developed Characters**: The characters in the movie are well-fleshed out, with distinct personalities and motivations. Cobb's backstory and emotional arc are particularly well-developed, making him a relatable and sympathetic protagonist.
3. **Stunning Visuals and Action Sequences**: The movie's visuals are breathtaking, with impressive action sequences that showcase the team's skills and creativity. The hotel corridor fight scene and the zero-gravity hotel sequence are standouts.
4. **Thought-Provoking Themes**: The movie explores themes of reality, identity, and the power of the human mind, leaving the audience to ponder the implications 

In [ ]:
# Example of a Parallel Chain

blog_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are blog post writer."),
        ("human","Give me a brief summary about the topic {topic_name}")
    ]
)

def twitter_blog(t_blog):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are blog post writer."),
            ("human", "Analyze the blog: {t_blog}. Write a twitter post on this topic."),
        ]
    )
    return plot_template.format_prompt(t_blog = t_blog)

def facebook_blog(f_blog):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are blog post writer."),
            ("human", "Analyze the blog: {f_blog}. Write a facebook post on this topic."),
        ]
    )
    return plot_template.format_prompt(f_blog = f_blog)

def linkedin_blog(l_blog):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are blog post writer."),
            ("human", "Analyze the blog: {l_blog}. Write a LinkedIn post on this topic."),
        ]
    )
    return plot_template.format_prompt(l_blog = l_blog)

def combine_all(t_blog, f_blog, l_blog):
  return f"Twitter Blog:\n{t_blog}\n\nFacebook blog:\n{f_blog}\n\nLinkedIn blog:\n{l_blog}"

twitter_blog_chain = (RunnableLambda(lambda x: twitter_blog(x)) | llm | StrOutputParser())
facebook_blog_chain = (RunnableLambda(lambda x: facebook_blog(x)) | llm | StrOutputParser())
linkedin_blog_chain = (RunnableLambda(lambda x: linkedin_blog(x)) | llm | StrOutputParser())

chain = (
    blog_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches = {"t_blog":twitter_blog_chain,"f_blog":facebook_blog_chain,"l_blog":linkedin_blog_chain})
    | RunnableLambda(lambda x: combine_all(x["branches"]["t_blog"],x["branches"]["f_blog"],x["branches"]["l_blog"]))
)

result = chain.invoke({"topic_name":"India"})
print(result)


Twitter Blog:
"Discover the vibrant country of India! With a rich history, diverse culture, and breathtaking natural beauty, India has something for everyone. From the Taj Mahal to the beaches of Goa, explore the many wonders of this incredible nation #India #Travel #Culture"

Facebook blog:
Here's a Facebook post based on the blog:

**Discover the Wonders of India!**

Did you know that India is the world's seventh-largest country by land area and the second-most populous country, with over 1.3 billion people? 

From the majestic Himalayan Mountains to the beautiful beaches of Goa, India is a country of incredible diversity and natural beauty. With over 22 official languages and a significant presence of various religions, India is a melting pot of cultures and traditions.

Whether you're interested in history, culture, food, or adventure, India has something for everyone! Explore the iconic landmarks like the Taj Mahal, the Golden Temple, and the Red Fort, or experience the vibrant cu

#Conditional Chain

In [5]:
# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | llm | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | llm | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | llm | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | llm | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

# Output the result
print(result)

Here's a potential response to address the negative feedback:

"Dear valued customer,

We apologize for the extremely disappointing experience you've had with our product. We're truly sorry to hear that it broke after only one use and that the quality did not meet your expectations. We understand that this is not the level of quality and performance you deserve, and for that, we are truly sorry.

At our company, we take pride in providing high-quality products that meet the needs of our customers. Clearly, we fell short in your case, and we want to make it right. We would like to offer you a replacement or a full refund, whichever you prefer. Please let us know which option you prefer, and we will expedite the process.

Additionally, we would like to learn more about what went wrong with your product. Your feedback is invaluable in helping us identify areas for improvement and ensuring that our products meet the high standards we strive for. If you're willing, could you please provide 